In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import sys
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
trains = pd.read_pickle("all_trains.pickle")

In [5]:
trains["train_id"] = trains[["origin", "destination", "departure","arrival"]].sum(axis=1)

In [6]:
columns_to_select = [
    "train_id",
    "departure",
    "arrival",
    "duration",
    "vehicle_class",
    "Turista_Promo +_seats",
    "Turista_Promo +_price",
    "insert_date",
    "timedelta_till_dep",
    "origin",
    "destination",
]
df = trains[columns_to_select]

In [37]:
trains["vehicle_class"].value_counts()

Turista         2383553
Turista Plus     244727
Preferente       209424
Name: vehicle_class, dtype: int64

In [35]:
df["vehicle_class"].unique()

array(['Turista'], dtype=object)

In [36]:
df = df[df["vehicle_class"] == "Turista"]

In [11]:
df["weekday"] = df["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").weekday()
)
df["depart_month"] = df["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").month
)
df["depart_hour"] = df["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour
)

In [12]:
df["depart_day"] = df["departure"].apply(
    lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").day
)

In [13]:
def stripstring(x):
    x = x[:10]
    return x

In [14]:
df["insert_date_2"] = df["insert_date"].apply(lambda x: stripstring(x))

In [16]:
df["insert_date_month"] = df["insert_date_2"].apply(
    lambda x: str(datetime.datetime.strptime(x, "%Y-%m-%d").month)
)
df["insert_date_day"] = df["insert_date_2"].apply(
    lambda x: str(datetime.datetime.strptime(x, "%Y-%m-%d").day)
)

In [17]:
def insert_month_day(month,day):
    if len(day) == 1:
        day = "0" + day
    return month + day

In [18]:
def add_0(day):
    if len(day) == 1:
        day = "0" + day
    return day

In [19]:
insert_month_day("2", "10")

'210'

In [20]:
df["insert_date_day"] = df["insert_date_day"].apply(add_0)


In [21]:
df["insert_date_day"].unique()

array(['29', '30', '31', '01', '02', '03', '04', '05', '06', '07', '08',
       '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27', '28'], dtype=object)

In [22]:
df["insert_month_day"] = df["insert_date_month"] + df["insert_date_day"]
df["insert_month_day"].unique()

0          129
1          129
2          129
3          129
4          129
          ... 
2837699    331
2837700    331
2837701    331
2837702    331
2837703    331
Name: insert_month_day, Length: 2383553, dtype: object

In [24]:
df.columns

Index(['train_id', 'departure', 'arrival', 'duration', 'vehicle_class',
       'Turista_Promo +_seats', 'Turista_Promo +_price', 'insert_date',
       'timedelta_till_dep', 'origin', 'destination', 'weekday',
       'depart_month', 'depart_hour', 'depart_day', 'insert_date_2',
       'insert_date_month', 'insert_date_day', 'insert_month_day'],
      dtype='object')

In [25]:
data = (
    df.groupby(["train_id","insert_month_day"])
    .agg(
        mean_econ_price=("Turista_Promo +_price", np.nanmean),
        max_econ_seats=("Turista_Promo +_seats", max),
        min_econ_seats=("Turista_Promo +_seats", min),
        mean_econ_seats=("Turista_Promo +_seats", "mean"),
        depart_month=("depart_month","first"),
        depart_day=("depart_day", "first"),
        depart_hour=("depart_hour", "first"),
        insert_date = ("insert_date_2", "first"),
        departure = ("departure","first"),
        origin = ("origin","first"),
        destination = ("destination","first"),
    )
    .reset_index()
)

In [27]:
data.shape

(259763, 13)

In [28]:
data["train_id"].nunique()

10590

In [29]:
data = data[data['mean_econ_seats'].notna()]

In [30]:
data["train_id"].nunique()

7596

In [31]:
data["insert_month_day"].unique()

array(['221', '222', '223', '225', '226', '227', '228', '229', '301',
       '302', '303', '304', '305', '306', '307', '308', '309', '310',
       '311', '312', '313', '314', '315', '220', '206', '207', '208',
       '209', '210', '211', '212', '213', '214', '215', '216', '217',
       '218', '219', '224'], dtype=object)

In [33]:
data.head(2)

,train_id,insert_month_day,mean_econ_price,max_econ_seats,min_econ_seats,mean_econ_seats,depart_month,depart_day,depart_hour,insert_date,departure,origin,destination
0,ALBACETEMADRID2020-03-02 08:05:002020-03-02 09...,221,38.2,171.0,171.0,171.000000,3,2,8,2020-02-21,2020-03-02 08:05:00,ALBACETE,MADRID
1,ALBACETEMADRID2020-03-02 08:05:002020-03-02 09...,222,38.2,171.0,166.0,168.181818,3,2,8,2020-02-22,2020-03-02 08:05:00,ALBACETE,MADRID


In [40]:
data["price_change"] = (
    data.sort_values(by=["train_id", "insert_date"], ascending=True)
    .groupby("train_id")["mean_econ_price"]
    .transform("diff")
)

data["price_change_percentage"] = (data["price_change"] / data["mean_econ_price"]).round(3)

data["price_change_direction"] = np.where(
    data["price_change_percentage"] > 0,
    1,
    np.where(data["price_change_percentage"] < 0, -1, 0),
)

data["price_increase"] = np.where(data["price_change_direction"] == 1, True, False)
data["price_decrease"] = np.where(data["price_change_direction"] == -1, True, False)

C:\Users\Velkey Artúr\AppData\Local\Temp\ipykernel_4796\3963774062.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["price_change"] = (
C:\Users\Velkey Artúr\AppData\Local\Temp\ipykernel_4796\3963774062.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["price_change_percentage"] = (data["price_change"] / data["mean_econ_price"]).round(3)
C:\Users\Velkey Artúr\AppData\Local\Temp\ipykernel_4796\3963774062.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [41]:
data["seats_sold"] = abs((
    data.sort_values(by=["train_id", "insert_date"], ascending=True)
    .groupby("train_id")["mean_econ_seats"]
    .transform("diff")
))

C:\Users\Velkey Artúr\AppData\Local\Temp\ipykernel_4796\4190282701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["seats_sold"] = abs((


In [42]:
data["price_increase"].value_counts()

False    151918
True      14131
Name: price_increase, dtype: int64

In [43]:
data["depart_monthday"] = data["departure"].apply(lambda date: date[5:10])

C:\Users\Velkey Artúr\AppData\Local\Temp\ipykernel_4796\4007202796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["depart_monthday"] = data["departure"].apply(lambda date: date[5:10])


In [44]:
dfs = [
    x
    for _, x in data[data["insert_month_day"] != "206"].groupby(
        ["insert_month_day", "depart_monthday", "origin", "destination"]
    )
]

In [48]:
from tqdm.notebook import tqdm

In [49]:
for i in tqdm(dfs):
    number_lags = 1
    for lag in range(1, number_lags + 1):
        i["seats_sold_lag" + str(lag)] = i.seats_sold.shift(lag)
        i["pair_" + str(lag)] = i.train_id.shift(lag)

    number_lags_back = 1
    for lag in range(1, number_lags_back + 1):
        i["seats_sold_lag_backward" + str(lag)] = i.seats_sold.shift(-lag)
        i["pair_2"] = i.train_id.shift(-lag)

    i["prop_values"] = i["seats_sold"] / (
        i["seats_sold"]
        + i["seats_sold_lag1"].replace(np.nan, 0)
        + i["seats_sold_lag_backward1"].replace(np.nan, 0)
    )

  0%|          | 0/22335 [00:00<?, ?it/s]

In [50]:
teljes_data = pd.concat(dfs)

In [51]:
teljes_data.shape

(163373, 25)

In [117]:
teljes_data.to_csv("data_with_prop_ext.csv")